# **Data Pipeline**

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.5 MB/s 
     |████████████████████████████████| 101 kB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 27.7 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install plotly
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import regex as re
import transformers
#import keras
from tensorflow.keras import backend as K




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/ML Project/mbti_1.csv')
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
#Check if TPU is available
use_tpu = True
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.88.203.2:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.88.203.2:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.88.203.2:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


**Data is extremely imbalanced.** This might cause overfitting to happen since the total data only amounts to 8675.

In [ ]:
data['type'].value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [ ]:
def clean_text(data):
    data_length=[]
    lemmatizer=WordNetLemmatizer()
    cleaned_text=[]
    for sentence in tqdm(data.posts):
        sentence=sentence.lower()
        
        #removing links from text data
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
        #removing other symbols
        sentence=re.sub('[^0-9a-z]',' ',sentence)
    
        
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text

In [ ]:
data.posts = clean_text(data)
data

  0%|          | 0/8675 [00:00<?, ?it/s]

,type,posts
0,INFJ,and intj moments sportscenter not top t...
1,ENTP,i m finding the lack of me in these posts ver...
2,INTP,good one course to which i say i ...
3,INTJ,dear intp i enjoyed our conversation the o...
4,ENTJ,you re fired that s another silly misconce...
...,...,...
8670,ISFP,just because i always think of cats as fi d...
8671,ENFP,so if this thread already exists someplace ...
8672,INTP,so many questions when i do these things i ...
8673,INFP,i am very conflicted right now when it comes ...


**Initialize BERT Tokenizer and attention masks******

In [ ]:
#Split dataset
from sklearn.model_selection import train_test_split

posts = data['posts'].values
labels =  data['type'].values
train_data, test_data = train_test_split(data, random_state=0, test_size=0.2)

train_size = len(train_data)
test_size = len(test_data)
train_size, test_size

(6940, 1735)

In [ ]:
#Initialize Bert tokenizer and masks
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 512

def tokenize_sentences(sentences, tokenizer, max_seq_len = 1800):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)
        
    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

train_input_ids = tokenize_sentences(train_data['posts'], tokenizer, MAX_LEN)
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_attention_masks = create_attention_masks(train_input_ids)

test_input_ids = tokenize_sentences(test_data['posts'], tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

  0%|          | 0/6940 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/1735 [00:00<?, ?it/s]

In [ ]:
#train_masks,test_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.2)

In [ ]:
#Create train and test datasets
BATCH_SIZE=32 
NR_EPOCHS=10
#def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
#    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
#    if train:
#        dataset = dataset.shuffle(buffer_size=buffer_size)
#    dataset = dataset.repeat(epochs)
#    dataset = dataset.batch(batch_size)
#    if train:
#        dataset = dataset.prefetch(1)
    
 #   return dataset

#train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
#test_dataset = create_dataset((test_inputs, test_masks, test_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE, train=False)

# BERT Model
* Load the pretrained BERT base-model from Transformers library
- Take the first hidden-state from BERT output (corresponding to CLS token) and feed it into a Dense layer with 16 neurons and softmax activation

In [ ]:

#from transformers import TFBertModel

#from tensorflow.keras.layers import Dense, Flatten

#class BertClassifier(tf.keras.Model):    
#        def __init__(self, bert: TFBertModel, num_classes: int):
#            super().__init__()
#            self.bert = bert
#            self.classifier = Dense(16, activation='softmax')

#        @tf.function
#        def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
#            outputs = self.bert(input_ids,
#                                   attention_mask=attention_mask,
#                                   token_type_ids=token_type_ids,
#                                   position_ids=position_ids,
#                                   head_mask=head_mask)
#            cls_output = outputs[1]
#            cls_output = self.classifier(cls_output)

#            return cls_output
        
        
#with strategy.scope():        
#    model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols))

In [ ]:
#Define f1 functions for evaluation
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
def create_model(): 
    input_word_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32,
                                           name="input_word_ids")
    bert_layer = transformers.TFBertModel.from_pretrained('bert-large-uncased')
    bert_outputs = bert_layer(input_word_ids)[0]
    pred = tf.keras.layers.Dense(16, activation='softmax')(bert_outputs[:,0,:])
    
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=pred)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00002), metrics=['accuracy', f1_m, precision_m, recall_m])
    return model

In [ ]:
def get_callbacks():
    
    best_model = tf.keras.callbacks.ModelCheckpoint(
        f'best_model.h5',
        verbose=1, 
        monitor='val_loss', 
        mode='min', 
        save_best_only=True, 
        save_weights_only=True
    )
    
    last_model = tf.keras.callbacks.ModelCheckpoint(
        f'last_model.h5',
        verbose=1, 
        save_best_only=False, 
        save_weights_only=True
    )
    
    callbacks = [best_model , last_model]
    
    return callbacks

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.88.203.2:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.88.203.2:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 512)]            0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  335141888
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             512, 1024),                         
                              pooler_output=(None, 10            
                             24),                                
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [ ]:
types = np.unique(data.type.values)

def get_type_index(string):
    return list(types).index(string)

In [ ]:
train_data['type_index'] = data['type'].apply(get_type_index)
train_data

,type,posts,type_index
984,INTP,phrases i never want to hear again a k a if yo...,11
6706,INTP,yeah you say you primarily value people who ...,11
143,ENFP,63915 i got my hair cut d some people say t...,1
4844,INFP,as far as i live in this world i ve never bee...,9
4388,ISFP,meh it s overplayed ya but still its good n...,13
...,...,...,...
4373,INFP,hey it seems like you have a great foundatio...,9
7891,INFJ,dear istj mother when i started my very fi...,8
4859,INTP,oh entjs how can you be scary and exciting a...,11
3264,ENFJ,hi entp and welcome to the forum wink f...,0


In [ ]:
one_hot_labels = tf.keras.utils.to_categorical(train_data.type_index.values, num_classes=16)

In [ ]:

model.fit(np.array(train_input_ids), one_hot_labels, verbose = 1, epochs = NR_EPOCHS, batch_size = BATCH_SIZE,callbacks=get_callbacks(), validation_split=0.25)

Epoch 1/10


163/163 [==============================] - ETA: 0s - loss: 2.0922 - accuracy: 0.3404 - f1_m: 0.1723 - precision_m: 0.2923 - recall_m: 0.1304
Epoch 1: val_loss improved from inf to 1.62501, saving model to best_model.h5

Epoch 1: saving model to last_model.h5
163/163 [==============================] - 390s 1s/step - loss: 2.0922 - accuracy: 0.3404 - f1_m: 0.1723 - precision_m: 0.2923 - recall_m: 0.1304 - val_loss: 1.6250 - val_accuracy: 0.5308 - val_f1_m: 0.4630 - val_precision_m: 0.6663 - val_recall_m: 0.3750
Epoch 2/10
163/163 [==============================] - ETA: 0s - loss: 1.4500 - accuracy: 0.5710 - f1_m: 0.5102 - precision_m: 0.7182 - recall_m: 0.4201
Epoch 2: val_loss improved from 1.62501 to 1.32086, saving model to best_model.h5

Epoch 2: saving model to last_model.h5
163/163 [==============================] - 102s 629ms/step - loss: 1.4500 - accuracy: 0.5710 - f1_m: 0.5102 - precision_m: 0.7182 - recall_m: 0.4201 - val_loss: 1.3209 - val_accuracy: 0.6104 - val_f1_m: 0.6097 -

In [43]:
import pickle
pickle.dump(model, open('Personality_Prediction.pkl','wb'))

INFO:tensorflow:Assets written to: ram://1d688296-3a69-41d4-9383-1ff47c25e5e2/assets


INFO:tensorflow:Assets written to: ram://1d688296-3a69-41d4-9383-1ff47c25e5e2/assets


In [ ]:
train_data['type'].value_counts()

INFP    1476
INFJ    1175
INTP    1027
INTJ     852
ENTP     547
ENFP     536
ISTP     275
ISFP     224
ENTJ     196
ISTJ     181
ENFJ     147
ISFJ     131
ESTP      73
ESFP      40
ESTJ      31
ESFJ      29
Name: type, dtype: int64

In [ ]:
test_data['type'].value_counts()

INFP    356
INFJ    295
INTP    277
INTJ    239
ENFP    139
ENTP    138
ISTP     62
ISFP     47
ENFJ     43
ISFJ     35
ENTJ     35
ISTJ     24
ESTP     16
ESFJ     13
ESFP      8
ESTJ      8
Name: type, dtype: int64

**Run test and evaluate accuracy**

In [42]:
test_data['type_index'] = data['type'].apply(get_type_index)
test_data

,type,posts,type_index
4587,ISFP,dear isfj mother i wish you were less of a w...,13
2786,INFJ,to me i think you guys may be over analyzing...,8
2813,ENFP,nihm while nihm has her intj husband i ve go...,1
3705,INTP,i want 5 kids an astro nuclear theoretical...,11
5957,ISFP,i have the same thing as well i ve noticed t...,13
...,...,...,...
418,INFJ,yes i do it all the time i just throw up my...,8
6712,ISFP,hah yes that would be fantastic d d...,13
6390,ISFP,the fact that she s an intj too pretty much e...,13
4925,INFJ,to be perfectly honest i don t know why you ...,8


In [ ]:
test_labels = tf.keras.utils.to_categorical(test_data.type_index.values, num_classes=16)

In [ ]:
model.evaluate(np.array(test_input_ids), test_labels)

55/55 [==============================] - 12s 159ms/step - loss: 2.1399 - accuracy: 0.6265 - f1_m: 0.6278 - precision_m: 0.6576 - recall_m: 0.6074


[2.1399285793304443,
 0.6265130043029785,
 0.6277596950531006,
 0.657575786113739,
 0.6073863506317139]

**The accuracy is not great, this is because the data inside the dataset is very imbalanced, which causes the huge disparation between training score and testing score (overfitting).**

In [ ]:
cols = data['type'].unique()
cols = cols.tolist()

colnames = ['sentence']
colnames = colnames+cols


In [ ]:

df_predict = pd.DataFrame(columns = colnames)
sentence = ["'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin*  That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This + Lack of Balance and Hand Eye Coordination.|||Real IQ test I score 127. Internet IQ tests are funny. I score 140s or higher.  Now, like the former responses of this thread I will mention that I don't believe in the IQ test. Before you banish...|||You know you're an ENTP when you vanish from a site for a year and a half, return, and find people are still commenting on your posts and liking your ideas/thoughts. You know you're an ENTP when you...|||http://img188.imageshack.us/img188/6422/6020d1f9da6944a6b71bbe6.jpg|||http://img.adultdvdtalk.com/813a0c6243814cab84c51|||I over think things sometimes. I go by the old Sherlock Holmes quote.  Perhaps, when a man has special knowledge and special powers like my  own, it rather encourages him to seek a complex...|||cheshirewolf.tumblr.com  So is I :D|||400,000+  post|||Not really; I've never thought of E/I or J/P as real functions.  I judge myself on what I use. I use Ne and Ti as my dominates. Fe for emotions and rarely Si. I also use Ni due to me strength...|||You know though. That was ingenious. After saying it I really want to try it and see what happens with me playing a first person shooter in the back while we drive around. I want to see the look on...|||out of all of them the rock paper one is the best. It makes me lol.  You guys are lucky :D I'm really high up on the tumblr system.|||So did you hear about that new first person shooter game? I've been rocking the hell out of the soundtrack on my auto sound equipment that will shake the heavens. We managed to put a couple PS3's in...|||No; The way he connected things was very Ne. Ne dominates are just as aware of their environments as Se dominates.  Example: Shawn Spencer or Patrick Jane; Both ENTPs.|||Well charlie I will be the first to admit I do get jealous like you do. I chalk it up to my 4w3 heart mixed with my dominate 7w8. 7s and 8s both like to be noticed. 4's like to be known (not the same...|||;D I'll upload the same clip with the mic away from my mouth. Than you won't hear anything.  Ninja Assassin style but with splatter.|||Tik Tok is a really great song. As long as you can mental block out the singer. I love the beat it makes me bounce.|||drop.io v1swck0  :D Mic really close to my mouth and smokin aces: assassins ball playing in the background.|||Sociable =/= extrovert; I'm an extrovert and I'm not sociable. :)|||Sherlock in the movie was an ENTP. Normally he's played as a EXTJ. In the books he's an ESTJ.  As I said. The movie looked good except for it being called sherlock holmes.|||http://i817.photobucket.com/albums/zz96/kamioo/Dirtywinch.png|||Oh, I never had fear of kissing a guy. I will kiss an animal too. So there was nothing to vanish. Just personal taste and me not liking it.  The guy I kissed didn't know me. It was one of those...|||Sounds pretty much like my area and what I'm going through right now trying to figure out which way I want to take my life. I want to do so many things. The biggest problem is that I know if I don't...|||;D I was operating under the impression that you were female. I never looked at your boxy. Okay, I help out my gay friends all the time and one of them has developed a little crush on me. I get red...|||T_T You just described me  and I'm living the worst nightmare. I'm trapped in one place with one one around. Only dull woods. If I was a serial killer this would be the perfect place but sadly I'm...|||TBH, and biased, sounds like a shadowed INFP. I think maybe he was hurt and turned ESTJ. I can tell because he has some of the typical INFP traits left over.|||*Checks list* I'm sorry. It seems that you have came at a bad time. We've already reached our quota of INFJs. However, being you're female and I like females I will make you a deal. I will kick one...|||I'm ANTP (Leaning toward E). I'm easy for both ENTPs and INTPs to identify with. :)|||I also imagine ENTP's interrogations would go a little bit like Jack's from 24 except more mechanical. Rigging up shock treatment equipment in an abandoned building out of an old car batty, jumper...|||It was a compliment :) Trust me. I'm just as psychopathic :D except I have emoticons. They're just weird ones. Like laughing when I get hurt or at people running themselves over with their lawn mower...|||http://i817.photobucket.com/albums/zz96/kamioo/Thunderstorm.pnghttp://i817.photobucket.com/albums/zz96/kamioo/Thunderstormbw.png http://i817.photobucket.com/albums/zz96/kamioo/Cosmicstorm.png|||No. It's like a theme for where I live and that is why I know it by heart.   http://www.youtube.com/watch?v=j5W73HaVQBg|||and I usual don't leave until the thing ends. But in the mean time. In between times. You work your thing. I'll work mine :D  ;D I'm the MBP; Pleasure to meet you.|||Damn, need to trust my instincts more I would have been closer I was going to say INFP.|||EXFP? Leaning toward S with the way she responded.  :D My friends, even my gay and lesbian ones, always come to me for advice.|||I bow to my entp masters ENTPs are so great. If it wasn't for ENTPs I wouldn't have been able to build what I'm building  Duck Duck  Duck  Shotgun|||What? Me? I never do that >.> <.<|||Because its hard to be sad about losing someone you like when you knew you were right and give yourself a big pat on the back because you're awesome and always correct.|||Oh, you don't have to tell me that most of them are stupid. I know this. That is why I play with them and it makes me laugh. :D As I'm going to take Neuropsychology and I have a few psychologist...|||:D I'm a Nightowl. I wake up between 6-7pm and stay awake till 10-11:30am.|||Personal opinion backed by theory would suggest that INTPs are the most socially difficult. While INTJs can be socially indifferent but they will also use social situations if the the need arises....|||Personal stocks that I have on my desktop that I've downloaded from random stock sites and stock photobuckets.|||I'll tell you when I open photoshop.  :) Glad you like it static.|||:D Thanks.|||http://i817.photobucket.com/albums/zz96/kamioo/Deathgrip.png http://i817.photobucket.com/albums/zz96/kamioo/Deathgripbw.png  Made for a friend. Several hours of work. I constructed every line by...|||:) Static: http://i817.photobucket.com/albums/zz96/kamioo/Statickitten.png  I'll have to get to your avatar later if one of my fellow teammates doesn't.|||Psychologist don't keep me around long enough to diagnosis me. I like to toy with them. What I have diagnosis myself with and had a few psychologist friends (+ a few other friends) tell me I have is...'"]
sentence_inputs = tokenize_sentences(sentence, tokenizer, MAX_LEN)
sentence_inputs = pad_sequences(sentence_inputs, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
sentence_inputs

prediction = model.predict(np.array(sentence_inputs))
df_predict.loc[0,'sentence']=sentence
df_predict.loc[0, cols] = prediction

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sentence_inputs

array([[  101,  1005,  1045,  1005,  1049,  4531,  1996,  3768,  1997,
         2033,  1999,  2122,  8466,  2200,  8598,  2075,  1012,  1064,
         1064,  1064,  3348,  2064,  2022, 11771,  2065,  2009,  1005,
         1055,  1999,  1996,  2168,  2597,  2411,  1012,  2005,  2742,
         2033,  1998,  2026,  6513,  2024,  2747,  1999,  2019,  4044,
         2073,  2057,  2031,  2000,  5541,  2135,  2224, 11190, 15239,
         1998,  8696,  1012,  2045,  3475,  1005,  1056,  2438,  1012,
         1012,  1012,  1064,  1064,  1064,  3228,  2047,  3574,  2000,
         1005,  2208,  1005,  3399,  1012,  1064,  1064,  1064,  7592,
         1008,  4372, 25856,  5861,  1008,  2008,  1005,  1055,  2035,
         2009,  3138,  1012,  2084,  2057, 23705,  1998,  2027,  2079,
         2087,  1997,  1996, 20661,  2096,  1045, 13399,  2037,  3739,
         1998,  2709,  2037,  2616,  2007,  5744,  2773, 13068,  1998,
         2062,  5048,  2100, 20237,  1012,  1064,  1064,  1064,  2023,
      

In [ ]:
df_predict

,sentence,INFJ,ENTP,INTP,INTJ,ENTJ,ENFJ,INFP,ENFP,ISFP,ISTP,ISFJ,ISTJ,ESTP,ESFP,ESTJ,ESFJ
0,['I'm finding the lack of me in these posts ve...,0.0,0.000014,0.00001,0.999531,0.00001,0.000007,0.000006,0.000023,0.000006,0.000001,0.000005,0.000373,0.000002,0.000001,0.000009,0.000002


In [ ]:
sentence_inputs = tokenize_sentences(df_predict['sentence'], tokenizer, MAX_LEN)
sentence_inputs = pad_sequences(sentence_inputs, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
prediction = model.predict(np.array(sentence_inputs))
df_predict.loc[0, cols] = prediction

df_predict

  0%|          | 0/1 [00:00<?, ?it/s]

,sentence,INFJ,ENTP,INTP,INTJ,ENTJ,ENFJ,INFP,ENFP,ISFP,ISTP,ISFJ,ISTJ,ESTP,ESFP,ESTJ,ESFJ
0,['I'm finding the lack of me in these posts ve...,0.031535,0.638181,0.000589,0.002056,0.009441,0.020855,0.0151,0.023456,0.00043,0.000937,0.007116,0.007636,0.027976,0.047422,0.004492,0.162779


**Test the model to predict a single sentence. 